# Emotion Detection Model Training
## Google Colab Notebook

This notebook trains a CNN model for facial emotion recognition using the FER2013 dataset.

**Features:**
- GPU acceleration
- Data augmentation
- Advanced callbacks (early stopping, learning rate scheduling)
- Real-time training visualization
- Model export for download

**Detected Emotions:** Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q deeplake tensorflow opencv-python matplotlib

# Check GPU availability
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Import libraries
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import deeplake
import os
from datetime import datetime
from IPython.display import clear_output

print("All libraries imported successfully!")

## 2. Load Dataset

In [ ]:
# Load FER2013 dataset from DeepLake
print("Loading datasets...")
train_ds = deeplake.load('hub://activeloop/fer2013-train')
val_ds = deeplake.load('hub://activeloop/fer2013-public-test')

print(f"Training samples: {len(train_ds):,}")
print(f"Validation samples: {len(val_ds):,}")

# Emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
print(f"\nEmotion classes: {emotion_labels}")

## 3. Visualize Sample Data

In [ ]:
# Visualize some samples
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i in range(10):
    sample = train_ds[i * 1000]
    image = sample.images.data()["value"]
    label = sample.labels.data()["value"]
    
    axes[i].imshow(image, cmap='gray')
    axes[i].set_title(f"{emotion_labels[label]}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 4. Data Preprocessing and Augmentation

In [ ]:
def augment_image(image):
    """Apply data augmentation to an image."""
    # Random horizontal flip
    if np.random.random() > 0.5:
        image = cv2.flip(image, 1)
    
    # Random rotation (-15 to 15 degrees)
    if np.random.random() > 0.5:
        angle = np.random.uniform(-15, 15)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    
    # Random brightness adjustment
    if np.random.random() > 0.5:
        brightness = np.random.uniform(0.7, 1.3)
        image = np.clip(image * brightness, 0, 255).astype(np.uint8)
    
    return image


def deeplake_generator(ds, batch_size, augment=False):
    """Generate batches of data from DeepLake dataset."""
    while True:
        batch_images = []
        batch_labels = []
        for sample in ds:
            image = sample.images.data()["value"]
            label = sample.labels.data()["value"]

            # Preprocess image
            image = cv2.resize(image, (48, 48))
            
            # Apply augmentation if enabled
            if augment:
                image = augment_image(image)
            
            # Normalize and add channel dimension
            image = np.expand_dims(image, axis=-1) / 255.0

            # One-hot encode the label
            label = tf.keras.utils.to_categorical(label, num_classes=7)
            label = np.squeeze(label)

            batch_images.append(image)
            batch_labels.append(label)

            # Yield the batch if it's full
            if len(batch_images) == batch_size:
                yield np.array(batch_images), np.array(batch_labels)
                batch_images, batch_labels = [], []

        # Handle the remaining samples in the last batch
        if batch_images:
            yield np.array(batch_images), np.array(batch_labels)

print("Data generators defined!")

## 5. Define Model Architecture

In [ ]:
def create_model():
    """Create the emotion detection CNN model with batch normalization."""
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(48, 48, 1)),
        
        # First convolutional block
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.25),

        # Second convolutional block
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.25),

        # Dense layers
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(7, activation='softmax')
    ])
    
    return model

# Create and display model
model = create_model()
model.summary()

## 6. Configure Training

In [ ]:
# Training parameters
BATCH_SIZE = 64
NUM_EPOCHS = 100
INITIAL_LR = 0.001

# Create data generators
train_generator = deeplake_generator(train_ds, BATCH_SIZE, augment=True)
validation_generator = deeplake_generator(val_ds, BATCH_SIZE, augment=False)

# Calculate steps
steps_per_epoch = len(train_ds) // BATCH_SIZE
validation_steps = len(val_ds) // BATCH_SIZE

print(f"Training Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Steps per epoch: {steps_per_epoch}")
print(f"  Validation steps: {validation_steps}")
print(f"  Initial learning rate: {INITIAL_LR}")

# Compile model
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=INITIAL_LR),
    metrics=['accuracy']
)

print("\nModel compiled successfully!")

## 7. Setup Callbacks

In [ ]:
# Create callbacks
callbacks = [
    # Save best model
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_emotion_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=False,
        mode='max',
        verbose=1
    ),
    
    # Early stopping
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate on plateau
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Callbacks configured!")

## 8. Train Model

**Note:** This will take some time. With GPU, expect ~2-3 hours for 100 epochs.

In [ ]:
# Train the model
print("Starting training...")
print("="*60)

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=NUM_EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*60)
print("Training completed!")
print("="*60)

## 9. Visualize Training Results

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot accuracy
axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].legend(loc='best')
axes[0].grid(True, alpha=0.3)

# Plot loss
axes[1].plot(history.history['loss'], label='Train', linewidth=2)
axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].legend(loc='best')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Print final metrics
print("\nFinal Metrics:")
print(f"  Training Accuracy: {history.history['accuracy'][-1]:.4f}")
print(f"  Validation Accuracy: {history.history['val_accuracy'][-1]:.4f}")
print(f"  Training Loss: {history.history['loss'][-1]:.4f}")
print(f"  Validation Loss: {history.history['val_loss'][-1]:.4f}")

## 10. Export Model

In [ ]:
# Save model weights (compatible with your application)
model.save_weights('model.weights.h5')
print("Model weights saved as 'model.weights.h5'")

# Save full model
model.save('emotion_model_full.h5')
print("Full model saved as 'emotion_model_full.h5'")

print("\n✅ Training complete! Download the files to use in your application.")

## 11. Download Files

Download the trained model files to your local machine:

In [ ]:
from google.colab import files

# Download model weights
print("Downloading model.weights.h5...")
files.download('model.weights.h5')

# Download full model (optional)
print("Downloading emotion_model_full.h5...")
files.download('emotion_model_full.h5')

# Download training plot
print("Downloading training_history.png...")
files.download('training_history.png')

print("\n✅ All files downloaded!")

## Usage Instructions

1. Download `model.weights.h5` from the files above
2. Place it in your project directory (same folder as `main.py`)
3. Run your emotion music player application

```bash
python main.py
```

---

**Happy Training! 🎉**